In [16]:
import os
import glob
import numpy as np
from scipy.io.matlab import loadmat
from nilearn import plotting
from nilearn import image

In [2]:
data_path = '/home/remi/data/AV_att/'
dropbox_path = '/home/remi/Dropbox/PhD/Experiments/AV_Integration_7T/Subjects_Data'

In [3]:
conditions_names = [
    ' A Stim - Auditory Attention',
    ' V Stim - Auditory Attention',
    ' AV Stim - Auditory Attention',
    ' A Stim - Visual Attention',
    ' V Stim - Visual Attention',
    ' AV Stim - Visual Attention']

In [4]:
subj_ls = glob.glob(os.path.join(data_path, 'sub*'), recursive=False)

In [8]:
for i_subj in subj_ls:
    
    subj = i_subj.split('/')[-1]
    
    print(subj[4:])
    
    # open the SPM.mat and look for the name of the regressors
    SPM_mat = os.path.join(dropbox_path, 'Subject_' + subj[4:], 'FFX_Block', 'SPM.mat')
    SPM_mat = loadmat(SPM_mat, struct_as_record=True)
    
    SPM_xX = SPM_mat['SPM'][0][0][7] # that would be SPM.xX (I think)
    reg_names = SPM_xX[0][0][6][0] # that should be all the regressor names
    
    nb_reg = reg_names.size
    
    cdt_vec = []
    
    # we loop through all the regressor names and check to which condition each belong
    for i_reg in range(nb_reg):

        reg = reg_names[i_reg][0] # extract the string from the array

        cdt_vec.append(-1); # dummy values to ignore this regressor in case it is not waht we want

        # we loop through the condition names and check that we have a match 
        # (also with bf(1) to make sure it is the HRF contrast)
        for i_cdt in range(len(conditions_names)):
            # if there is match we give that regressor the number of the condition
            if reg.find(conditions_names[i_cdt]) != -1 and reg.find('bf(1)') != -1:
                cdt_vec[i_reg] = i_cdt 
                break
    
    
    for i_cdt in range(len(conditions_names)):
        # find indices of beta images that correspond to each condition
        ind_list = [i for i, value in enumerate(cdt_vec) if value == i_cdt] 
        
        beta_img = []
        
        for i_beta in ind_list:
            file_name = "r4beta_{0:04d}.nii.gz".format(i_beta + 1)
            beta_img.append( os.path.join(i_subj, 'Betamapping', file_name) )
            
            #exists = os.path.isfile(beta_img)
            #if exists:
            
            #beta_img_hdr = nib.load(beta_img)
            #beta_img_data = beta_img_hdr.get_fdata()
            

    


12
09
08
03
02
11
15
13
07
04
16


In [ ]:
ROI = '/home/remi/Dropbox/PhD/Experiments/AV_Integration_7T/ROI/ret_proba_atlas/maxprob_vol_lh.nii.gz'

plotting.plot_glass_brain(ROI)   
#plotting.plot_stat_map(ROI)
plotting.show()

In [ ]:
data = image.load_img(beta_img)

In [20]:
image_data = data.get_fdata()

In [21]:
type(image_data)

numpy.ndarray

In [19]:
image_data

AttributeError: 'numpy.ndarray' object has no attribute 'dim'

In [22]:
# We first create a masker, giving it the options that we care
# about. Here we use standardizing of the data, as it is often important
# for decoding
from nilearn.input_data import NiftiMasker
#masker = NiftiMasker(mask_img=mask_filename, standardize=True)

# We give the masker a filename and retrieve a 2D array ready
# for machine learning with scikit-learn
#fmri_masked = masker.fit_transform(fmri_filename)

In [23]:
help(NiftiMasker)

Help on class NiftiMasker in module nilearn.input_data.nifti_masker:

class NiftiMasker(nilearn.input_data.base_masker.BaseMasker, nilearn._utils.cache_mixin.CacheMixin)
 |  NiftiMasker(mask_img=None, sessions=None, smoothing_fwhm=None, standardize=False, detrend=False, low_pass=None, high_pass=None, t_r=None, target_affine=None, target_shape=None, mask_strategy='background', mask_args=None, sample_mask=None, dtype=None, memory_level=1, memory=Memory(location=None), verbose=0)
 |  
 |  Applying a mask to extract time-series from Niimg-like objects.
 |  
 |  NiftiMasker is useful when preprocessing (detrending, standardization,
 |  resampling, etc.) of in-mask voxels is necessary. Use case: working with
 |  time series of resting-state or task maps.
 |  
 |  Parameters
 |  ----------
 |  mask_img : Niimg-like object, optional
 |      See http://nilearn.github.io/manipulating_images/input_output.html
 |      Mask for the data. If not given, a mask is computed in the fit step.
 |      Opt